In [2]:
import numpy as np

import pandas as pd


In [5]:
n_decimals = 3
n_iters = 100

n_obs = 10**4
n_categories = 10**3

first_category = np.random.choice(n_categories, n_obs)
np.random.seed(2016)
y = np.random.normal(0, 1, n_obs)

df1 = pd.DataFrame({'first category': first_category, 
                   'y': y})
df2 = pd.DataFrame({'first category': first_category, 
                   'y': y})

In [6]:
%%timeit
group_means = df1.groupby('first category').mean()


1.09 ms ± 6.83 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
%%timeit
group_means = df2.groupby('first category').mean()


1.1 ms ± 6.35 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [8]:
import functools

In [9]:
df['mean'] = df.groupby('first category')['y'].transform(np.mean)
print(df)

NameError: name 'df' is not defined

In [ ]:
import time

n_decimals = 3
n_iters = 100

n_obs = 10**4
n_categories = 10**3

first_category = np.random.choice(n_categories, n_obs)
np.random.seed(2016)
y = np.random.normal(0, 1, n_obs)

df = pd.DataFrame({'first category': first_category,
                   'y': y})
                     
start = time.process_time()
grouped = df.groupby('first category')
pandas_answer = grouped['y'].apply(np.mean)
print('time to compute group means once with Pandas: {0}'\
      .format(round(time.process_time() - start, n_decimals)))

start =  time.process_time()
for i in range(n_iters):
    grouped['y'].apply(np.mean)
print('time to compute group means {0} times with Pandas: {1}'\
      .format(n_iters, round(time.process_time() - start, n_decimals)))

In [ ]:
class Groupby:
    def __init__(self, keys):
        _, self.keys_as_int = np.unique(keys, return_inverse = True)
        self.n_keys = max(self.keys_as_int) + 1
        self.set_indices()
        
    def set_indices(self):
        self.indices = [[] for i in range(self.n_keys)]
        for i, k in enumerate(self.keys_as_int):
            self.indices[k].append(i)
        self.indices = [np.array(elt) for elt in self.indices]
        
    def apply(self, function, vector):
        result = np.zeros(self.n_keys)
        for k, idx in enumerate(self.indices):
            result[self.keys_as_int[k]] = function(vector[idx])
        return result
    
    def transform(self, function, vector):
        result = np.zeros(len(vector))
        for idx in self.indices:
            result[idx] = function(vector[idx])
        return result

In [ ]:
unique_keys, indices = np.unique(['a', 'b', 'a', 'a', 'b'], return_inverse = True)
# array([{'b', 'a'}], dtype=object)
print(unique_keys) 
# [0, 1, 0, 0, 1]
print(indices) 
# array(['a', 'b', 'a', 'a', 'b'], dtype='<U1')
print(unique_keys[indices]) 

In [ ]:
start = time.process_time()
grouped = Groupby(first_category)

group_means = Groupby(first_category).apply(np.mean, y)
print('time to compute group means once with Grouped: {0}'\
      .format(round(time.process_time() - start, n_decimals)))

start = time.process_time()
grouped = Groupby(first_category)
for i in range(n_iters):
    grouped.apply(np.mean, y)
    
print('time to compute group means {0} times with Grouped: {1}'\
      .format(n_iters, round(time.process_time() - start, n_decimals)))

In [ ]:
grouped.apply(np.mean, y)

In [ ]:
import pandas as pd
from joblib import Parallel, delayed
import multiprocessing

def tmpFunc(df):
    x=df.b
    res=(x.iloc[-1]-x) / np.timedelta64(1, 's')
    return res

def applyParallel(dfGrouped, func):
    retLst = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(func)(group) for name, group in dfGrouped)
    return pd.concat(retLst)

df = pd.DataFrame({'a': first_category,
                   'b': pd.to_datetime(y).values})
print('parallel version: ')
%time print(applyParallel(df.groupby('a'), tmpFunc))

print('regular version: ')
%time print(df.groupby('a').apply(tmpFunc))



In [ ]:
import pandas as pd
from joblib import Parallel, delayed
import multiprocessing

def tmpFunc(df):
    df['c'] = df.a + df.b
    return df

def applyParallel(dfGrouped, func):
    retLst = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(func)(group) for name, group in dfGrouped)
    return pd.concat(retLst)



In [ ]:
class Groupby:
    def __init__(self, keys):
        _, self.keys_as_int = np.unique(keys, return_inverse = True)
        self.n_keys = max(self.keys_as_int) + 1
        self.set_indices()
        
        
    def set_indices(self):
        self.indices = [[] for i in range(self.n_keys)]
        for i, k in enumerate(self.keys_as_int):
            self.indices[k].append(i)
        self.indices = [np.array(elt) for elt in self.indices]
        
    def apply(self, function, vector):
        result = np.zeros(self.n_keys)
        for k, idx in enumerate(self.indices):
            result[self.keys_as_int[k]] = function(vector[idx])
        return result
    
    def transform(self, function, vector):
        result = np.zeros(len(vector))
        n_jobs=multiprocessing.cpu_count()
        partial=functools.partial(function,vector)
        retLst = pool.map(partial, self.indices)
        return pd.concat(retLst)

In [ ]:

n_decimals = 3
n_iters = 100

n_obs = 10**6
n_categories = 10**5

first_category = np.random.choice(n_categories, n_obs)
np.random.seed(2016)
y = np.random.normal(0, 1, n_obs)
df1 = pd.DataFrame({'a': first_category,
                  'b': pd.to_datetime(y).values,
                    'c': pd.to_datetime(y).values})
df2 = pandas.DataFrame({'a': first_category,
                   'b': pd.to_datetime(y).values,
                    'c': pd.to_datetime(y).values})

In [ ]:
def h(vec,e):
    x=vec[e]
    return (x.iloc[-1]-x) / np.timedelta64(1, 's')

In [ ]:
%time res=(df1['b'].iloc[-1]-df1['b'].iloc[0]) / np.timedelta64(1, 's')

In [ ]:
%time res=df1.groupby('a')['b'].transform(lambda x:(x.iloc[-1]-x) / np.timedelta64(1, 's'))

In [ ]:
%time res=df2['b'].apply(lambda x:(df2['b'].iloc[-1]-x) / np.timedelta64(1, 's'))

In [ ]:
res

In [ ]:
%time res=(df1['b'].iloc[-1]-df1['b']) / np.timedelta64(1, 's')
res1

In [ ]:
nrows, ncols = 20000, 100

df1, df2, df3, df4 = [pandas.DataFrame(np.random.randn(nrows, ncols)) for _ in range(4)]

In [ ]:
np.random.seed(0)
df1 = pd.DataFrame(np.random.choice(10, (5, 4)), columns=list('ABCD'))
df2 = pd.DataFrame(np.random.choice(10, (5, 4)), columns=list('ABCD'))
df3 = pd.DataFrame(np.random.choice(10, (5, 4)), columns=list('ABCD'))
df4 = pd.DataFrame(np.random.choice(10, (5, 4)), columns=list('ABCD'))

In [ ]:

x = 5
%timeit pd.eval("df1.A + (df1.B * x)")  

In [ ]:
%timeit df1.A + (df1.B * x)


In [ ]:
%timeit pd.eval('df1 + df2 + df3 + df4')

In [ ]:
a=df1['b'].iloc[-1]
s=df1['b']
b=np.timedelta64(1, 's')
pandas.eval(s+s)

In [ ]:
def func(x):
    return (x.iloc[-1]-x) / np.timedelta64(1, 's')

In [ ]:
print('modin version: ')
%time tmp1=df1.groupby('a')['b'].transform(func).values
tmp1.shape

In [ ]:
tmp1.shape

In [ ]:

print('own version')
%time tmp2=df2.groupby('a').transform(lambda x: (x.iloc[-1]-x) / np.timedelta64(1, 's')).values
np.array_equal(tmp1, tmp2)

In [ ]:
functools.partial(h,df.b)

In [ ]:
def sum2(x, y):
    return x + y

incr2 = functools.partial(sum2, 1)
incr2(4)


In [ ]:
np.array_equal(tmp1, tmp2.sort_index())

In [ ]:
tmp1

In [ ]:
import pandas as pd
import itertools
import time
import multiprocessing
from typing import Callable, Tuple, Union

def groupby_parallel(groupby_df: pd.core.groupby.DataFrameGroupBy,
                     func: Callable[[Tuple[str, pd.DataFrame]], Union[pd.DataFrame, pd.Series]],
                     num_cpus: int=multiprocessing.cpu_count() - 1,
                     logger: Callable[[str], None]=print) -> pd.DataFrame:
    """Performs a Pandas groupby operation in parallel.
    Example usage:
        import pandas as pd
        df = pd.DataFrame({'A': [0, 1], 'B': [100, 200]})
        df.groupby(df.groupby('A'), lambda row: row['B'].sum())
    Authors: Tamas Nagy and Douglas Myers-Turnbull
    """
    start = time.time()
    logger("\nUsing {} CPUs in parallel...".format(num_cpus))
    with multiprocessing.Pool(num_cpus) as pool:
        queue = multiprocessing.Manager().Queue()
        result = pool.starmap_async(func, [(name, group) for name, group in groupby_df])
        cycler = itertools.cycle('\|/―')
        while not result.ready():
            logger("Percent complete: {:.0%} {}".format(queue.qsize()/len(groupby_df), next(cycler)), end="\r")
            time.sleep(0.4)
        got = result.get()
    logger("\nProcessed {} rows in {:.1f}s".format(len(got), time.time() - start))

In [ ]:
df = pd.DataFrame({'A': [0, 1], 'B': [100, 200]})
df.groupby('A').apply( lambda row: row['B'].sum())

In [ ]:
groupby_parallel(df.groupby('A'), lambda row: row['B'].sum())

In [ ]:

import multiprocessing

def my_function(x):
    """The function you want to compute in parallel."""
    x += 1
    return x


if __name__ == '__main__':
    pool = multiprocessing.Pool()
    results = pool.map(my_function, [1,2,3,4,5,6])
    print(results)

In [ ]:
df = pandas.DataFrame({'a': np.random.randn(1000),
                   'b': np.random.randn(1000),
                   'N': np.random.randint(100, 1000, (1000)),
                   'x': 'x'})

In [ ]:
def f(x):
    return x * (x - 1)

def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx


In [ ]:
%timeit df.apply(lambda x: integrate_f(x['a'], x['b'], x['N']), axis=1)

In [ ]:
%prun -l 4 df.apply(lambda x: integrate_f(x['a'], x['b'], x['N']), axis=1) 

In [ ]:
df = pd.DataFrame({'a': np.random.randn(1000),
                   'b': np.random.randn(1000),
                   'N': np.random.randint(100, 1000, (1000)),
                   'x': 'x'})

In [ ]:
%timeit df.apply(lambda x: integrate_f(x['a'], x['b'], x['N']), axis=1)